In [ ]:
from soview import *
import shutil
import os
import gc

In [ ]:
import pandas as pd

In [ ]:
raw_path = 'zhiyuanyuan/PUBDT/st/slideDNAseq/original/data/ft_local/slide-DNA-seq'
h5ad_path = 'zhiyuanyuan/PUBDT/st/slideDNAseq/original/h5ad'
# sample = 'DNAseqFISH+1Mbloci-E14-clonal24hr'



In [ ]:
def get_adata(var_dict,count_file,delimiter,molecule):
    var_len = len(var_dict)
    count_pd = pd.read_csv(f'{raw_path}/{count_file}',delimiter=delimiter)
    count_sparse = count_pd.values
    if molecule=='dna':
        cell_len = count_sparse[:,0].max()
    elif molecule=='rna':
        cell_len = count_sparse[:,1].max()
    # print(cell_len,var_len)
    # print(count_sparse[:,1].max(),count_sparse[:,0].max())
    count_mat = np.zeros(shape=(cell_len,var_len))
    
    for i in range(count_sparse.shape[0]):
        if molecule=='dna':
            cell_idx = count_sparse[i][0]
            var_idx = count_sparse[i][1]
        elif molecule=='rna':
            cell_idx = count_sparse[i][1]
            var_idx = count_sparse[i][0]
        count = count_sparse[i][2]
        count_mat[cell_idx-1,var_idx-1] = count
    adata = ad.AnnData(count_mat)
    adata.var_names = [var_dict[i+1] for i in range(len(var_dict))]
    adata.obs_names = list(np.arange(cell_len).astype('str'))
    return adata

In [ ]:
for file_name in os.listdir(raw_path):
    file_sp = file_name.split('.')
    if len(file_sp)!=3:
        continue
    print(file_sp[0],file_sp[1])
        

In [ ]:
human_dna_var_file = f'{raw_path}/hg19_1Mb_bins.txt'
human_rna_var_file = f'{raw_path}/human_genes.txt'
mouse_dna_var_file = f'{raw_path}/mm10_1Mb_bins.txt'
mouse_rna_var_file = f'{raw_path}/mouse_genes.txt'

In [ ]:
pd_human_dna = pd.read_csv(human_dna_var_file,delimiter='\t')
human_dna_var_dict = {}
for i in range(len(pd_human_dna)):
    bin_id = pd_human_dna['bin_ind'][i]
    chr_id = pd_human_dna['chr'][i]
    bin_st = pd_human_dna['bin_start'][i]
    bin_ed = pd_human_dna['bin_end'][i]    
    human_dna_var_dict[bin_id] = f'{chr_id}_{bin_st}_{bin_ed}'

In [ ]:
pd_mouse_dna = pd.read_csv(mouse_dna_var_file,delimiter='\t')
mouse_dna_var_dict = {}
for i in range(len(pd_mouse_dna)):
    bin_id = pd_mouse_dna['bin_ind'][i]
    chr_id = pd_mouse_dna['chr'][i]
    bin_st = pd_mouse_dna['bin_start'][i]
    bin_ed = pd_mouse_dna['bin_end'][i]    
    mouse_dna_var_dict[bin_id] = f'{chr_id}_{bin_st}_{bin_ed}'

In [ ]:
pd_human_rna = pd.read_csv(human_rna_var_file,delimiter='\t')
rna_list = list(pd_human_rna.values[:,0])
id_list = list(np.arange(len(rna_list))+1)
human_rna_var_dict = dict(zip(id_list,rna_list))

In [ ]:
pd_mouse_rna = pd.read_csv(mouse_rna_var_file,delimiter='\t')
rna_list = list(pd_mouse_rna.values[:,0])
id_list = list(np.arange(len(rna_list))+1)
mouse_rna_var_dict = dict(zip(id_list,rna_list))

In [ ]:
sample_file_list = [
    [
        'human_colon_cancer_3_dna_191204_19.bead_locations.csv',
        'human_colon_cancer_3_dna_191204_19.sparse_counts_1Mb.txt'
    ],
    [
        'human_colon_cancer_4_dna_200114_13.bead_locations.csv',
        'human_colon_cancer_4_dna_200114_13.sparse_counts_1Mb.txt'
    ],
    [
        'human_colon_cancer_4_rna_200102_06.bead_locations.csv',
        'human_colon_cancer_4_rna_200102_06.sparse_expression.txt'
    ],
    [
        'human_colon_cancer_dna_4x_201027_12.bead_locations.csv',
        'human_colon_cancer_dna_4x_201027_12.sparse_counts_1Mb.txt'
    ],
    [
        'mouse_cerebellum_1_dna_190201_21.bead_locations.csv',
        'mouse_cerebellum_1_dna_190201_21.sparse_counts_1Mb.txt'
    ],
    [
        'mouse_cerebellum_1_dna_200114_14.bead_locations.csv',
        'mouse_cerebellum_1_dna_200114_14.sparse_counts_1Mb.txt'
    ],
    [
        'mouse_cerebellum_dna_4x_201027_17.bead_locations.csv',
        'mouse_cerebellum_dna_4x_201027_17.sparse_counts_1Mb.txt'
    ],
    [
        'mouse_liver_met_1_dna_191114_05.bead_locations.csv',
        'mouse_liver_met_1_dna_191114_05.sparse_counts_1Mb.txt'
    ],
    [
        'mouse_liver_met_1_dna_191114_06.bead_locations.csv',
        'mouse_liver_met_1_dna_191114_06.sparse_counts_1Mb.txt'
    ],
    [
        'mouse_liver_met_2_dna_200114_10.bead_locations.csv',
        'mouse_liver_met_2_dna_200114_10.sparse_counts_1Mb.txt'
    ],
    [
        'mouse_liver_met_2_rna_201002_04.bead_locations.csv',
        'mouse_liver_met_2_rna_201002_04.sparse_expression.txt'
    ]
    
]

In [ ]:
for i in range(len(sample_file_list)):
    count_file = sample_file_list[i][1]
    coord_file = sample_file_list[i][0]
    sample_name = coord_file.split('.')[0]
    if 'dna' in count_file and 'human' in count_file:
        var_dict = human_dna_var_dict
        delimiter = '\t'
        molecule = 'dna'
    if 'dna' in count_file and 'mouse' in count_file:
        var_dict = mouse_dna_var_dict
        delimiter = '\t'
        molecule = 'dna'


    if 'rna' in count_file and 'human' in count_file:
        var_dict = human_rna_var_dict
        delimiter = ','
        molecule = 'rna'


    if 'rna' in count_file and 'mouse' in count_file:
        var_dict = mouse_rna_var_dict        
        delimiter = ','
        molecule = 'rna'



    adata = get_adata(var_dict,count_file,delimiter,molecule)
    spatial_mat = coord_pd = pd.read_csv(f'{raw_path}/{coord_file}').values[:,1:3]
    adata.obsm['spatial'] = spatial_mat.astype('float')
    print(sample_name,adata.shape)
    

    adata.write_h5ad(f'{h5ad_path}/{sample_name}.h5ad')
    del adata
    gc.collect()

In [ ]:
h5ad_path

In [ ]:
from soview import *
import shutil
import os
import gc

root_path = '/aaa/jianhuayao/project2/zhiyuanyuan/projects/SODB/'
source_h5ad_path = h5ad_path
biotech_category = 'Spatial_Genomics'
biotech = 'Slide-DNA-seq'
# 多个单词要加下划线
dataset_short = 'zhao2022spatial'

h5ad_list = os.listdir(source_h5ad_path)
out_dataname_list = []
out_h5ad_path_list = []
out_feature_dict_list = []
N_Unit_list = []
N_Gene_list = []
for hl in h5ad_list:
    if not hl.endswith('.h5ad'):
        continue
    if hl=='HDST.h5ad':
        continue
    # source_h5ad_data = hl.split('.')[0]
    source_h5ad_data = hl[0:-5]


    source_h5ad = f'{source_h5ad_path}/{source_h5ad_data}.h5ad'

    data_name = source_h5ad_data
    
    import os

    path_rel = f'demo_data/{biotech_category}/{biotech}/{dataset_short}/{data_name}'
    path_abs = root_path+path_rel
    os.makedirs(path_abs, exist_ok=False)
    print("The new directory is created!")
    shutil.copyfile(source_h5ad, f'{path_abs}/data.h5ad')
    
    
    data_use = path_rel
    adata = ad.read_h5ad(f'{root_path}{data_use}/data.h5ad')
    print(adata.X)
    print(f'h5ad path: {data_use}/data.h5ad')
    
    adata.obsm['spatial'] = np.array(adata.obsm['spatial']).astype('float')
    
    adata.layers["counts"] = adata.X.copy()
    
#     # if np.sum(adata.X)==np.sum(adata.X.astype('int')):
    if True:


# #         # is all the element in adata.X are integer?
    
        sc.pp.normalize_total(adata, inplace=True)
        sc.pp.log1p(adata)
    if adata.shape[1]>=2000:
        sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)
    else:
        pass
    

    sc.pp.pca(adata)
    sc.pp.neighbors(adata)
    sc.tl.leiden(adata)
    
    sc.pl.embedding(adata,basis='spatial',color='leiden')
    sc.tl.umap(adata,n_components=3)
    SOViewer_plot(adata,embedding_use='X_umap',dot_size=10)
    print(f'View path: {data_use}/View_UMAP_LAB.pdf')
    
    gene_list = np.array(adata.var_names).tolist()
    obs_list = np.array(adata.obs.keys()).tolist()
    obsm_list = []
    for obsm in np.array(adata.obsm.keys()).tolist():
        for i in range(adata.obsm[obsm].shape[1]):
            cur_obsm = f'{obsm}@{i}'
            obsm_list.append(cur_obsm)
            
    feature_dict = {
        'gene':gene_list,
        'obs':obs_list,
        'obsm':obsm_list
    }
    
    import pickle
    with open(f'{root_path}{data_use}/feature_dict.pickle', 'wb') as handle:
        pickle.dump(feature_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
    
    adata.write_h5ad(f'{root_path}{data_use}/data.h5ad')
    out_dataname_list.append(source_h5ad_data)
    out_h5ad_path_list.append(f'{data_use}/data.h5ad')
    out_feature_dict_list.append(f'{data_use}/feature_dict.pickle')
    N_Unit_list.append(adata.shape[0])
    N_Gene_list.append(adata.shape[1])
    
    print(source_h5ad_data)
    print(f'h5ad path: {data_use}/data.h5ad')
    print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
    print(f'N_Unit: {adata.shape[0]}')
    print(f'N_Gene: {adata.shape[1]}')
    del adata
    gc.collect()

In [ ]:
for i in out_dataname_list:
    print(i)

In [ ]:
for i in out_h5ad_path_list:
    print(i)

In [ ]:
for i in out_feature_dict_list:
    print(i)

In [ ]:
for i in N_Unit_list:
    print(i)

In [ ]:
for i in N_Gene_list:
    print(i)